# [모듈 2.1] SageMaker 클러스터에서 훈련 (VPC에서 실행)

이 노트북은 아래의 작업을 실행 합니다.
- SageMaker Hosting Cluster 에서 훈련을 실행
- 훈련한 Job 이름을 저장 
    - 다음 노트북에서 모델 배포 및 추론시에 사용 합니다.
---

이전 노트북에서 변수 정의를 가져옵니다.
- dataset_location 변수에 저장된 S3 의 데이터 경로를 이용 합니다.

In [7]:
%store -r

SageMaker의 세션을 얻고, role 정보를 가져옵니다.
- 위의 두 정보를 통해서 SageMaker Hosting Cluster에 연결합니다.

In [2]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

# VPC_Mode를 True, False 선택
#### **[중요] VPC_Mode에서 실행시에 True로 변경해주세요**

In [4]:
VPC_Mode = True

In [5]:
from sagemaker.tensorflow import TensorFlow

def retrieve_estimator(VPC_Mode):
    if VPC_Mode:
        # VPC 모드 경우에 subnets, security_group을 기술 합니다.
        estimator = TensorFlow(base_job_name='cifar10',
                               entry_point='cifar10_keras_sm_tf2.py',
                               source_dir='training_script',
                               role=role,
                               framework_version='2.0.0',
                               py_version='py3',
                               script_mode=True,                       
                               hyperparameters={'epochs': 2},
                               train_instance_count=1, 
                               train_instance_type='ml.p3.8xlarge',
                               subnets = ['subnet-090c1fad32165b0fa','subnet-0bd7cff3909c55018'],
                               security_group_ids = ['sg-0f45d634d80aef27e']                                              
                              )        
    else:
        estimator = TensorFlow(base_job_name='cifar10',
                               entry_point='cifar10_keras_sm_tf2.py',
                               source_dir='training_script',
                               role=role,
                               framework_version='2.0.0',
                               py_version='py3',
                               script_mode=True,                       
                               hyperparameters={'epochs': 2},
                               train_instance_count=1, 
                               train_instance_type='ml.p3.8xlarge')
    return estimator

estimator = retrieve_estimator(VPC_Mode)

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


학습을 수행합니다. 이번에는 각각의 채널(`train, validation, eval`)에 S3의 데이터 저장 위치를 지정합니다.<br>
학습 완료 후 Billable seconds도 확인해 보세요. Billable seconds는 실제로 학습 수행 시 과금되는 시간입니다.
```
Billable seconds: <time>
```

참고로, `ml.p2.xlarge` 인스턴스로 5 epoch 학습 시 전체 6분-7분이 소요되고, 실제 학습에 소요되는 시간은 3분-4분이 소요됩니다.

In [6]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)})

2021-01-26 09:36:30 Starting - Starting the training job...
2021-01-26 09:36:55 Starting - Launching requested ML instancesProfilerReport-1611653789: InProgress
.........
2021-01-26 09:38:24 Starting - Preparing the instances for training......
2021-01-26 09:39:17 Downloading - Downloading input data
2021-01-26 09:39:17 Training - Downloading the training image.....2021-01-26 09:40:07,359 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-01-26 09:40:07,869 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "eval": "/opt/ml/input/data/eval",
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "model_dir": "s3://sagemaker-ap-

## training_job_name 저장

현재의 training_job_name을 저장 합니다.
- training_job_name을 에는 훈련에 관련 내용 및 훈련 결과인 **Model Artifact** 파일의 S3 경로를 제공 합니다.

In [10]:
train_job_name = estimator._current_job_name

In [13]:
%store train_job_name

Stored 'train_job_name' (str)
